<a href="https://colab.research.google.com/github/Reemaalt/Detection-of-Hallucination-in-Arabic/blob/main/AnswerClusterer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



1.   load generate answers
2.   Entailment model load //chose a model + the pipline work
3.   Clustering useing the ent model
4.   Save the output into a file





In [1]:
!huggingface-cli login
!pip install datasets
!pip install  transformers torch evaluate


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `week1 test` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `week1 t

In [2]:
import torch
import json
import pandas as pd
import os
import torch.nn.functional as F
from transformers import (
    ElectraForSequenceClassification,
    ElectraTokenizerFast,
    Trainer,
    TrainingArguments,
    pipeline,
    AutoTokenizer,
    AutoModelForSequenceClassification
)
from datasets import load_dataset
import numpy as np
from evaluate import load

In [14]:
#load functions

# Load XNLI dataset
print("Loading XNLI dataset...")
dataset = load_dataset("xnli", "ar")
print(f"Dataset loaded: {dataset}")

# Load AraELECTRA model and tokenizer
model_name = "aubmindlab/araelectra-base-discriminator"
tokenizer = ElectraTokenizerFast.from_pretrained(model_name)
model = ElectraForSequenceClassification.from_pretrained(
    model_name,
    num_labels=3  # entailment, neutral, contradiction
)



Loading XNLI dataset...
Dataset loaded: DatasetDict({
    train: Dataset({
        features: ['premise', 'hypothesis', 'label'],
        num_rows: 392702
    })
    test: Dataset({
        features: ['premise', 'hypothesis', 'label'],
        num_rows: 5010
    })
    validation: Dataset({
        features: ['premise', 'hypothesis', 'label'],
        num_rows: 2490
    })
})


Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at aubmindlab/araelectra-base-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Tokenizing datasets...


Map:   0%|          | 0/392702 [00:00<?, ? examples/s]

Map:   0%|          | 0/5010 [00:00<?, ? examples/s]

...


Model architecture:
ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(64000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
       

Epoch,Training Loss,Validation Loss,Accuracy
1,0.551300,0.587914,0.769880
2,0.426700,0.632867,0.769076


Epoch,Training Loss,Validation Loss,Accuracy
1,0.551300,0.587914,0.769880
2,0.426700,0.632867,0.769076
3,0.323000,0.748420,0.770683



Training results:
TrainOutput(global_step=73632, training_loss=0.4573476932609356, metrics={'train_runtime': 6944.4196, 'train_samples_per_second': 169.648, 'train_steps_per_second': 10.603, 'total_flos': 7.74938740264658e+16, 'train_loss': 0.4573476932609356, 'epoch': 3.0})

Model saved to ./araelectra-nli-finetuned

Testing fine-tuned model:


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [ ]:
#trinning functions
# Preprocess function with labels
def preprocess_function(examples):

    # Tokenize the premise and hypothesis pairs
    tokenized = tokenizer(
        examples["premise"],
        examples["hypothesis"],
        truncation=True,
        max_length=128,
        padding="max_length"
    )

    # Add labels to the tokenized output
    tokenized["labels"] = examples["label"]

    return tokenized

# Tokenize datasets
print("Tokenizing datasets...")
tokenized_datasets = dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=dataset["train"].column_names
)

# Print sample to verify data format
print("\nSample preprocessed data:")
print(tokenized_datasets["train"][0])

# Load accuracy metric
metric = load("accuracy")

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./araelectra-nli-finetuned",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    eval_strategy="epoch",        # Updated from evaluation_strategy
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
    report_to="wandb",           # Add WandB reporting
    logging_dir="./logs",
    logging_steps=500,
)

# Initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,         # Add tokenizer to trainer
)

# Print model architecture
print("\nModel architecture:")
print(model)

# Fine-tune the model
print("\nStarting training...")
train_result = trainer.train()

# Print training results
print("\nTraining results:")
print(train_result)

# Save the fine-tuned model
model_save_path = "./araelectra-nli-finetuned"
trainer.save_model(model_save_path)
tokenizer.save_pretrained(model_save_path)
print(f"\nModel saved to {model_save_path}")



In [18]:
#needed call
model = ElectraForSequenceClassification.from_pretrained("./araelectra-nli-finetuned")
tokenizer = ElectraTokenizerFast.from_pretrained("./araelectra-nli-finetuned")


In [19]:
# Test the fine-tuned model
def test_model(premise, hypothesis):
    inputs = tokenizer(
        premise,
        hypothesis,
        padding=True,
        truncation=True,
        max_length=128,
        return_tensors="pt"
    )

    outputs = model(**inputs)
    predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
    predicted_class = torch.argmax(predictions, dim=-1).item()

    label_map = {0: "entailment", 1: "neutral", 2: "contradiction"}
    return label_map[predicted_class], predictions[0].tolist()

# Test examples
test_examples = [
    ("القاهره عاصمه مصر", "القاهره هي عاصمة دولة مصر"),
    ("القاهره عاصمه مصر", "القاهره ليست عاصمة مصر"),
    ("القاهره عاصمه مصر", "مصر دولة عربية")
]

print("\nTesting fine-tuned model:")
for premise, hypothesis in test_examples:
    prediction, probabilities = test_model(premise, hypothesis)
    print(f"\nPremise: {premise}")
    print(f"Hypothesis: {hypothesis}")
    print(f"Prediction: {prediction}")
    print(f"Probabilities: entailment={probabilities[0]:.3f}, neutral={probabilities[1]:.3f}, contradiction={probabilities[2]:.3f}")


Testing fine-tuned model:

Premise: القاهره عاصمه مصر
Hypothesis: القاهره هي عاصمة دولة مصر
Prediction: entailment
Probabilities: entailment=0.943, neutral=0.050, contradiction=0.007

Premise: القاهره عاصمه مصر
Hypothesis: القاهره ليست عاصمة مصر
Prediction: contradiction
Probabilities: entailment=0.002, neutral=0.016, contradiction=0.982

Premise: القاهره عاصمه مصر
Hypothesis: مصر دولة عربية
Prediction: neutral
Probabilities: entailment=0.042, neutral=0.911, contradiction=0.047


In [20]:
#load generation file
file_path = '/content/generated_answers_Llama3.1-8b-xquad.json'

with open(file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

df = pd.DataFrame(data)
print(df.head())

   question_id                                           question  \
0          320                             ما هو المرفق بالسواكن؟   
1         1007     ما الدليل الذي تقدمه الجينات المُتَبَرَّع بها؟   
2          605  ماذا حدث في عام 1992 ضمن إجراءٍ على نطاق الممل...   
3          425                   متى أحدِثت توجيهية مجلس الأعمال؟   
4            4  من سجل أعلى عدد من الاستحواذات للفريق هذا الموسم؟   

                                     original_answer  \
0  {'text': ['غلاف التوربين'], 'answer_start': [1...   
1  {'text': ['وجود البلاستيدات الخضراء المفقود'],...   
2  {'text': ['أصبحت فيه كليات الفنون التطبيقية جا...   
3  {'text': ['استشارات الطاقة العمالية في الأعمال...   
4    {'text': ['كاوان شورت'], 'answer_start': [216]}   

                                   generated_answers  
0  [المرفق بالسواكن هو أحد الأوتار التالية للشخص ...  
1  [تتضمن الدنا العِرضيّة نطاقات الدنا المُتِبَرِ...  
2  [حصل على جائزة نوبل في الآداب وتم إشهار قانون ...  
3  [6/ 13/ 1433 هـ, أجرى مجل

In [21]:
# Define the entailment checking function BEFORE using it
def check_entailment_arabic(sentence1, sentence2):
    # Load the fine-tuned model

    model_path = "./araelectra-nli-finetuned"
    model = ElectraForSequenceClassification.from_pretrained(model_path)
    tokenizer = ElectraTokenizerFast.from_pretrained(model_path)

    # Move to GPU if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    # Prepare input
    inputs = tokenizer(
        sentence1,
        sentence2,
        padding=True,
        truncation=True,
        max_length=128,
        return_tensors="pt"
    )

    # Move inputs to device
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Get prediction
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probs = F.softmax(logits, dim=1)
        predicted_class = torch.argmax(probs, dim=1).item()

    # Map prediction to label
    return "entailment" if predicted_class == 0 else "neutral" if predicted_class == 1 else "contradiction"

def bidirectional_entailment(question, answer1, answer2):
    entailment1 = check_entailment_arabic(answer1, answer2)
    entailment2 = check_entailment_arabic(answer2, answer1)
    return entailment1 == "entailment" and entailment2 == "entailment"

# Clusters generated answers for a specific question based on bidirectional entailment.
def process_question(question_id, generated_answers):
    clusters = []  # List of clusters

   # Iterate over all generated answers
    for answer in generated_answers:
        added_to_cluster = False # Track whether the answer is added to a cluster

      # Compare the answer with representatives of existing clusters
        for cluster in clusters:
            representative = cluster[0] # Take the first answer as representative
            if bidirectional_entailment(question_id, representative, answer):
                cluster.append(answer) # Add the answer to the cluster
                added_to_cluster = True
                break
   # If the answer does not fit into any cluster, create a new cluster.
        if not added_to_cluster:
            clusters.append([answer])  # Create a new cluster
    return clusters



# Processes the entire dataset for entailment clustering.
def process_dataset(data):
    results = {}
    for question in data:
        question_id = question["question_id"]
        question_text = question["question"]
        generated_answers = question["generated_answers"]
        print(f"\nProcessing question {question_id}: {question_text}")
        clusters = process_question(question_id, generated_answers)
        results[question_id] = {
            "question": question_text,
            "clusters": clusters
        }
    return results

In [10]:

# Test the entailment function
response1 = "القاهره عاصمه مصر"
response2 = "القاهره عاصمة دولة مصر"
result = check_entailment_arabic(response1, response2)
print(f"Entailment test result: {result}")

Entailment test result: contradiction


In [22]:
# Apply the clustering to the loaded dataset
entailment_clusters = process_dataset(data)

# Output the results
for question_id, info in entailment_clusters.items():
    print(f"\nQuestion: {info['question']}")
    print(f"Question ID: {question_id}")
    for i, cluster in enumerate(info['clusters']):
        print(f"  Cluster {i + 1}: {cluster}")



Processing question 320: ما هو المرفق بالسواكن؟

Processing question 1007: ما الدليل الذي تقدمه الجينات المُتَبَرَّع بها؟

Processing question 605: ماذا حدث في عام 1992 ضمن إجراءٍ على نطاق المملكة المتحدة؟

Processing question 425: متى أحدِثت توجيهية مجلس الأعمال؟

Processing question 4: من سجل أعلى عدد من الاستحواذات للفريق هذا الموسم؟

Processing question 636: من صمم تصميم أيه بي سي الجديد لعام 1998؟

Processing question 631: ما جنسية فناني اللوحات التي تبرع بها جون شيبشانكس؟

Processing question 429: كم عدد الدول التي تحتوي "أمازونياس" في أسمائها؟

Processing question 647: ما هي نسبة مشاهدي التلفزيون في أوقات الذروة التي مثلتها الشبكات الثلاثة الكبرى في عام 1980؟

Processing question 286: أي زعيم للإصلاح السويسري كان من طلاب لوفيفر؟

Processing question 86: من بين سكان وارسو في عام 1901، كم كانت النسبة المئوية للكاثوليك؟

Processing question 226: ما الموسيقى التصويرية الاختيارية التي تتضمنها سكاي موفيز وسكاي بوكس أوفيس؟

Processing question 1098: لمَن كان إقبال ناقد؟

Processing qu

In [24]:
# Save results to a JSON file
output_file = "entailment_clusters_Llama3.1-8b-xquad.json"

with open(output_file, 'w', encoding='utf-8') as file:
    json.dump(entailment_clusters, file, ensure_ascii=False, indent=4)  # `ensure_ascii=False` keeps Arabic text readable

print(f"Results saved to {output_file}")


Results saved to entailment_clusters_Llama3.1-8b-xquad.json
